In [ ]:
#@title 1. Kütüphaneleri Yükle
# Gerekli kütüphaneleri yükler ve ortamı hazırlar.
# Bu hücreyi doğrudan çalıştırabilirsin.
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    import torch; v = re.match(r"[0-9\\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

In [ ]:
token=""
# Hugging Face Hub'a giriş yapmak için
from huggingface_hub import login
# Aşağıdaki "HF_TOKEN" yerine kendi kopyaladığın token'ı yapıştır
login(token=token)

In [ ]:
#@title 2. Modeli ve Tokenizer'ı Yükle
# Llama-3.2 11B Vision modelini 4-bit olarak yükler.
# Bu, daha az VRAM kullanmanı sağlar.
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True, # Bellek kullanımını azaltmak için 4-bit kullan. 16-bit LoRA için False yap.
    use_gradient_checkpointing = "unsloth",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.12: Fast Mllama patching. Transformers: 4.55.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: d1dd3ec5-e186-4678-80ab-a3aea0f5aff7)')' thrown while requesting HEAD https://huggingface.co/unsloth/llama-3.2-11b-vision-instruct-unsloth-bnb-4bit/resolve/4c5458f0d13e856b02c16daf7d39596070dfbe07/model-00001-of-00002.safetensors
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8d827991-027a-43b9-b41b-8c7b11736a70)')' thrown while requesting HEAD https://huggingface.co/unsloth/llama-3.2-11b-vision-instruct-unsloth-bnb-4bit/resolve/4c5458f0d13e856b02c16daf7d39596070dfbe07/model-00002-of-00002.safetensors
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 55d63398-d2be-46cf-99ca-bd4af34e417a)')' thrown while requesting HEAD https://huggingface.co/unsloth/llama-3.

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [ ]:
#@title 3. PEFT (LoRA) Adaptörlerini Ekle
# Verimli bir fine-tuning için LoRA adaptörleri eklenir.
# Hem görüntü hem de dil katmanlarını eğitmeyi seçiyoruz.
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # Görüntü katmanlarını fine-tune et
    finetune_language_layers   = True, # Dil katmanlarını fine-tune et
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,

    r = 32,
    lora_alpha = 64,
    lora_dropout = 0.05,
    bias = "none",
    random_state = 3407,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model.vision_model.transformer` require gradients


In [ ]:
from datasets import load_dataset

dataset_name = "itsanmolgupta/mimic-cxr-dataset"
dataset = load_dataset(dataset_name, split="train")

instruction = "You are an expert radiographer. Describe accurately what you see in this image."

def convert_to_conversation(sample):
    image = sample.get("image")

    # Bu dataset'te metin alanları 'impression' ve 'findings'
    caption = sample.get("impression") or sample.get("findings")

    # Boş/None ise atla
    if image is None or caption is None:
        return None
    if isinstance(caption, str) and caption.strip() == "":
        return None

    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": instruction},
                # HF Datasets 'Image' feature'ı PIL Image döndürür; kendi pipeline'ınız farklı bir anahtar istiyorsa burada uyarlayın
                {"type": "image", "image": image},
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": caption}],
        },
    ]
    return {"messages": conversation}

converted_dataset = [
    conv for sample in dataset
    if (conv := convert_to_conversation(sample)) is not None
]

print("Toplam örnek:", len(converted_dataset))
if converted_dataset:
    print(converted_dataset[0])


README.md:   0%|          | 0.00/357 [00:00<?, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/396M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/397M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/30633 [00:00<?, ? examples/s]

Toplam örnek: 30633
{'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'You are an expert radiographer. Describe accurately what you see in this image.'}, {'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7ECD144FE030>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'No acute cardiopulmonary process.'}]}]}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title 5. Modeli Eğit
# SFTTrainer ile modeli fine-tune ediyoruz.
# Bu işlem biraz zaman alabilir.
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) # Eğitim için modeli etkinleştir

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer), # Bunu kullanmak zorunlu!
    train_dataset = converted_dataset,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,

        num_train_epochs = 4,
        learning_rate = 1e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",

        # Vision fine-tuning için aşağıdaki parametreler zorunlu:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_length = 2048,
    ),
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 30,633 | Num Epochs = 4 | Total steps = 7,660
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 134,348,800 of 10,804,569,635 (1.24% trained)


Step,Training Loss
1,5.308200
2,5.239800
3,5.113500
4,4.095500
5,3.396600
6,3.158600
7,2.260200
8,2.002100
9,1.960000
10,1.438600


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
#@title 6. Sonuçları Test Et (Inference)
# Eğittiğimiz model ile bir örnek üzerinde deneme yapalım.
FastVisionModel.for_inference(model)

# Test için veri setinden bir görüntü alalım
image = dataset[10]["image"] # Farklı bir sonuç için indeksi (örn: 10) değiştirebilirsin
instruction = "You are an expert radiographer. Describe accurately what you see in this image."

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256) # max_new_tokens'ı artırabilirsin

The tip of the left PICC line is in the mid SVC.<|eot_id|>


In [ ]:
model.push_to_hub_merged(
    "Cosmobillian/radiologist_llama_v4",
    tokenizer,
    save_method = "merged_16bit", # VLLM gibi kütüphaneler için float16 formatında kaydet
    token = token
)

config.json: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...t_llama_v3/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0005-of-00005.safetensors:   2%|1         | 25.1MB / 1.47GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 5/5 [09:27<00:00, 113.51s/it]


Unsloth: Merge process complete. Saved to `/content/Cosmobillian/radiologist_llama_v3`
